In [1]:


# import utils.data_utils as du
from random import seed, random
import scipy.interpolate as si
from scipy.ndimage.interpolation import rotate
import numpy as np
import nibabel as nb
import glob
import matplotlib.pyplot as plt
import os
from config import settings
import re
import operator 

seed(1)
%matplotlib inline
%load_ext autoreload

%autoreload 1

In [2]:

ORIENTATION = {
    "COR": lambda x: x.transpose((0,2,1)),
    "AXI": lambda x: x,
    "SAG": lambda x: x.transpose((2,1,0)),
}

In [3]:
def save_nibabel_file(volume, header, filepath):
    mgh = nb.MGHImage(volume, np.eye(4), header)
    nb.save(mgh, 'processedData/' + filepath)
    print('file saved for ' + filepath)
    
def do_interpolate(values, steps, isLabel=False, target_steps=(2.0, 2.0, 3.0)):
    x, y, z = [steps[k] * np.arange(values.shape[k]) for k in range(3)]  # original grid
    if isLabel:
        method = 'nearest'
    else:
        method = 'linear'
    
    f = si.RegularGridInterpolator((x, y, z), values, method=method)  # interpolator

    dx, dy, dz = target_steps  # new step sizes # settings['EVAL']['target_voxel_dimension']
    new_grid = np.mgrid[0:x[-1]:dx, 0:y[-1]:dy, 0:z[-1]:dz]  # new grid
    new_grid = np.moveaxis(new_grid, (0, 1, 2, 3), (3, 0, 1, 2))  # reorder axes for evaluation
    return f(new_grid)

def load_data(file_path):
    volume_nifty, labelmap_nifty = nb.load(file_path[0]), nb.load(file_path[1])
    volume, labelmap = volume_nifty.get_fdata(), labelmap_nifty.get_fdata()
    return volume, labelmap, volume_nifty.header

def show(d_mat, l_mat, title='none'):
    print(title)
    shape = d_mat.shape
    idxs = shape[0]
    idx = 170
#     idx = np.int(random() * idxs)
#     idx = np.int(idxs/2)
    print('with shape:', shape, 'idx:', idx)
    plt.imshow(d_mat[idx], cmap='gray')
    plt.show()
    plt.imshow(l_mat[idx], cmap='gray')
    plt.show()
    
def rotate_orientation(volume_data, volume_label, orientation="AXI"):

    f_orient = ORIENTATION.get(orientation, 'not found!')

    if f_orient == 'not found!':
        raise ValueError("Invalid value for orientation. Pleas see help")

    return f_orient(volume_data), f_orient(volume_label)

def reduce_slices(data, labels, skip_Frame=1):
    """
    This function removes the useless black slices from the start and end. And then selects every even numbered frame.
    """
    no_slices, H, W  = data.shape
    l_slice, l_H, l_W = labels.shape

    if len(np.unique(labels)) != 3:
        raise Exception('all class are not present', np.unique(labels))

    if no_slices != l_slice or H != l_H or W != l_W:
        raise Exception('orig and seg shape does not match')

    mask_vector = np.zeros(no_slices, dtype=int)
    mask_vector[::3], mask_vector[1::3], mask_vector[2::3] = 1, 1, 0 
    mask_vector[:skip_Frame], mask_vector[-skip_Frame:-1] = 0, 0

    data_reduced = np.compress(mask_vector, data, axis=0).reshape(-1, H, W)
    labels_reduced = np.compress(mask_vector, labels, axis=0).reshape(-1, H, W)

    return data_reduced, labels_reduced


def remove_black(data, labels):
    clean_data, clean_labels = [], []
    for i, frame in enumerate(labels):
        unique, counts = np.unique(frame, return_counts=True)
        if counts[0] / sum(counts) < .99:
            clean_labels.append(frame)
            clean_data.append(data[i])
    return np.array(clean_data), np.array(clean_labels)

def cropND(img, bounding):
    start = list(map(lambda a, da: a//2-da//2, img.shape, bounding))
    end = list(map(operator.add, start, bounding))
    for i, val in enumerate(zip(start, end)):
        if val[0] < 0:
            start[i] = 0
            end[i] = img.shape[i]
    slices = tuple(map(slice, tuple(start), tuple(end)))
    return img[slices]


def post_interpolate(volume, labelmap, target_shape):
#     print('before crop:', volume.shape, labelmap.shape)
    volume = cropND(volume, target_shape)
    labelmap = cropND(labelmap, target_shape)
    current_shape = volume.shape
#     print('after crop:', volume.shape, labelmap.shape)
    intended_shape_deficit = target_shape - np.asarray(current_shape)

    paddings = [tuple(
        np.array([np.ceil((pad_tuples / 2) - pad_tuples % 2), np.floor((pad_tuples / 2) + pad_tuples % 2)]).astype(
            'int32')) for pad_tuples in intended_shape_deficit]
    paddings = tuple(paddings)

    volume = np.pad(volume, paddings, mode='constant')
    labelmap = np.pad(labelmap, paddings, mode='constant')
#     print('after pad:', volume.shape, labelmap.shape)

    return volume, labelmap

def find_nearest(source_num_arr, target_array=np.arange(0, 500, 16)):
    array = np.asarray(target_array)
    idxs = [(np.abs(array - source_num)).argmin() for source_num in source_num_arr]
    return [array[idx + 1] for idx in idxs]

def reorient(volume, labelmap, header, target_orientation=None):
    if target_orientation is None:
        target_orientation = [[0, 1],[1, 1],[2, 1]]
    source_orientation = nb.orientations.io_orientation(header.get_best_affine())
    transformation_mat = nb.orientations.ornt_transform(source_orientation, target_orientation)
    volume = nb.orientations.apply_orientation(volume, transformation_mat)
    labelmap = nb.orientations.apply_orientation(labelmap, transformation_mat)
    return volume, labelmap

def normalise_data(volume):
    volume = (volume - np.min(volume)) / (np.max(volume) - np.min(volume))
    return volume

def load_file_paths(data_dir=None, label_dir=None, volumes_txt_file=None, modality='1'):
    """
    This function returns the file paths combined as a list where each element is a 2 element tuple, 0th being data and 1st being label.
    It should be modified to suit the need of the project
    :param data_dir: Directory which contains the data files
    :param label_dir: Directory which contains the label files
    :param volumes_txt_file: (Optional) Path to the a csv file, when provided only these data points will be read
    :return: list of file paths as string
    """

    # Dataset dependent fetch configurations read from dataset config file.
    DATA_FETCH_CONFIGURATIONS = settings.get('DATA_FETCH_CONFIGURATIONS')

    DATA_CONFIG = settings.get('EVAL')

    if data_dir is None:
        data_dir = DATA_CONFIG['data_dir']
    if label_dir is None:
        label_dir = DATA_CONFIG['label_dir']
    
#     volumes_txt_file = DATA_CONFIG['volumes_txt_file']
    
    # Being used internally and different for different dataset.
    modality_map = eval(DATA_FETCH_CONFIGURATIONS['__modality_map__'])

    if volumes_txt_file:
        with open(volumes_txt_file) as file_handle:
            volumes_to_use = file_handle.read().splitlines()
    else:
        volumes_to_use = [name for name in os.listdir(data_dir)]

    new_excluded = []
    zero_file = []
    file_paths = []
    print('Toatl file to process:', len(volumes_to_use))
    for vol in volumes_to_use:
        try:
            _data_file_path_ = DATA_FETCH_CONFIGURATIONS['__data_file_path__']
            data_file_path = _data_file_path_.format(data_dir, vol, modality_map[modality])
            _label_file_path_ = DATA_FETCH_CONFIGURATIONS['__label_file_path__']
            label_file_path = _label_file_path_.format(label_dir, vol)

            files = glob.glob(eval(data_file_path))
            file_filtration_criterion = np.array([re.findall(r'\d+', f)[-1] for f in files], dtype='int')
            file_idx = np.where(file_filtration_criterion == file_filtration_criterion.min())
            file = files[file_idx[0][0]]
            file = [file]
            
            if len(file) is not 0:
                file_paths.append([file[0], eval(label_file_path)])
            else:
                zero_file.append(str(vol))
        except Exception as e:
            new_excluded.append([str(vol), str(e)])
            print(e)
            continue
            
    print(zero_file)
    print(new_excluded)
    np.savetxt('/home/abhijit/Jyotirmay/thesis/punet/processedData/zero_sets_round2.txt', np.asarray(zero_file), fmt='%s')
    np.savetxt('/home/abhijit/Jyotirmay/thesis/punet/processedData/excluded_round2.txt',  np.asarray(new_excluded), fmt='%s')
    
    return file_paths

def load_dataset(file_paths,
                 orientation,
                 reduce_slices=False,
                 remove_black=False):
    print("Loading and preprocessing data...")
    volume_list, labelmap_list = [], []
    rogue_vols = []
    for file_path in file_paths:
        try:
            volume, labelmap, header = load_and_preprocess(file_path, orientation,is_reduce_slices=reduce_slices,
                                                                                   is_remove_black=remove_black)

            filename = file_path[0].split('/')[-2]
            save_nibabel_file(volume, header, 'data/'+filename+'.nii.gz')
            save_nibabel_file(labelmap, header, 'labels/'+filename+'.nii.gz')
            volume_list.append(volume)
            labelmap_list.append(labelmap)
        
        except Exception as e:
            print(e)
            rogue_vols.append([str(file_path[0].split('/')[-2]), str(e)])
            print('rogue volume: ', file_path)
            continue

        finally:
            print("#", end='', flush=True)
            
    np.savetxt('/home/abhijit/Jyotirmay/thesis/punet/processedData/rogue_vol_round2.txt',  np.asarray(rogue_vols), fmt='%s')
    print(rogue_vols)
    print("100%", flush=True)
    return volume_list, labelmap_list

target_dim = [256, 144, 176]
def load_and_preprocess(file_path, orientation, is_reduce_slices=False, is_remove_black=False):
    global target_dim
    volume, labelmap, header = load_data(file_path)
    print('original', volume.shape, labelmap.shape)
    volume, labelmap = reorient(volume, labelmap, header)
    print('reorient', volume.shape, labelmap.shape)
    steps = header['pixdim'][1:4]
    volume = do_interpolate(volume, steps)

    labelmap = do_interpolate(labelmap, steps, True)
    print('interpolate', volume.shape, labelmap.shape)
    td = find_nearest(volume.shape)
    if target_dim is None:
        target_dim = td
    print(steps, target_dim)
    volume, labelmap = post_interpolate(volume, labelmap, target_shape=target_dim)
    print('post interpolate', volume.shape, labelmap.shape)
    volume, labelmap = rotate_orientation(volume, labelmap, orientation)
    print('rotate orient',volume.shape, labelmap.shape)
    labelmap = np.moveaxis(labelmap, 2, 0)
    volume = np.moveaxis(volume, 2, 0)
    print('move axis',volume.shape, labelmap.shape)
    if is_reduce_slices:
        volume, labelmap = reduce_slices(volume, labelmap)
    if is_remove_black:
        volume, labelmap = remove_black(volume, labelmap)
    volume = normalise_data(volume)

    
    return volume, labelmap, header

In [4]:
# DATA_FETCH_CONFIGURATIONS = settings.get('DATA_FETCH_CONFIGURATIONS')
file_paths = load_file_paths()
volume_list, labelmap_list = load_dataset(file_paths, 'AXI', reduce_slices=True, remove_black=True)

Toatl file to process: 385
{'1': 'opp_', '4': '[F_1|W_3|opp_8|in_6]', '2': '[F_1|W_3]'} /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2457401/*opp_[0-9]*.nii.gz /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2457401/comp_mask.nii [8 9] ['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2457401/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nii.gz']
{'1': 'opp_', '4': '[F_1|W_3|opp_8|in_6]', '2': '[F_1|W_3]'} /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2453015/*opp_[0-9]*.nii.gz /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2453015/comp_mask.nii [8 9] ['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2453015/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nii.gz']
{'1': 'opp_', '4': '[F_1|W_3|opp_8|in_6]', '2': '[F_1|W_3]'} /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2456340/*opp_[0-9]*.nii.gz /home/abhijit/nas_drive/Dat

{'1': 'opp_', '4': '[F_1|W_3|opp_8|in_6]', '2': '[F_1|W_3]'} /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2453251/*opp_[0-9]*.nii.gz /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2453251/comp_mask.nii [8 9] ['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2453251/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nii.gz']
{'1': 'opp_', '4': '[F_1|W_3|opp_8|in_6]', '2': '[F_1|W_3]'} /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2456535/*opp_[0-9]*.nii.gz /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2456535/comp_mask.nii [8 9] ['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2456535/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nii.gz']
zero-size array to reduction operation minimum which has no identity
{'1': 'opp_', '4': '[F_1|W_3|opp_8|in_6]', '2': '[F_1|W_3]'} /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2455946/*opp

{'1': 'opp_', '4': '[F_1|W_3|opp_8|in_6]', '2': '[F_1|W_3]'} /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2455727/*opp_[0-9]*.nii.gz /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2455727/comp_mask.nii [8 9] ['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2455727/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nii.gz']
{'1': 'opp_', '4': '[F_1|W_3|opp_8|in_6]', '2': '[F_1|W_3]'} /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2453427/*opp_[0-9]*.nii.gz /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2453427/comp_mask.nii [8 9] ['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2453427/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nii.gz']
{'1': 'opp_', '4': '[F_1|W_3|opp_8|in_6]', '2': '[F_1|W_3]'} /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2458158/*opp_[0-9]*.nii.gz /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/

{'1': 'opp_', '4': '[F_1|W_3|opp_8|in_6]', '2': '[F_1|W_3]'} /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA24531078/*opp_[0-9]*.nii.gz /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA24531078/comp_mask.nii [8 9] ['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA24531078/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nii.gz']
{'1': 'opp_', '4': '[F_1|W_3|opp_8|in_6]', '2': '[F_1|W_3]'} /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2453290/*opp_[0-9]*.nii.gz /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2453290/comp_mask.nii [8 9] ['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2453290/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nii.gz']
{'1': 'opp_', '4': '[F_1|W_3|opp_8|in_6]', '2': '[F_1|W_3]'} /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2459548/*opp_[0-9]*.nii.gz /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_se

{'1': 'opp_', '4': '[F_1|W_3|opp_8|in_6]', '2': '[F_1|W_3]'} /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2459745/*opp_[0-9]*.nii.gz /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2459745/comp_mask.nii [8 9] ['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2459745/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nii.gz']
{'1': 'opp_', '4': '[F_1|W_3|opp_8|in_6]', '2': '[F_1|W_3]'} /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2455268/*opp_[0-9]*.nii.gz /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2455268/comp_mask.nii [ 8  9 17 18] ['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2455268/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nii.gz']
{'1': 'opp_', '4': '[F_1|W_3|opp_8|in_6]', '2': '[F_1|W_3]'} /home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2460371/*opp_[0-9]*.nii.gz /home/abhijit/nas_drive/Data_WholeBody/KORA/KO

...

#original (288, 288, 160) (293, 429, 162)
reorient (288, 160, 288) (293, 162, 429)
interpolate (245, 136, 163) (249, 137, 243)
[1.7013888 1.7013888 1.699997 ] [256, 144, 176]
post interpolate (256, 144, 176) (260, 145, 189)
rotate orient (256, 144, 176) (260, 145, 189)
move axis (176, 256, 144) (189, 260, 145)
orig and seg shape does not match
rogue volume:  ['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2461206/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9.nii.gz', '/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2461206/comp_mask.nii']
#original (288, 288, 176) (288, 288, 176)
reorient (288, 176, 288) (288, 176, 288)
interpolate (245, 149, 163) (245, 149, 163)
[1.7013888 1.7013888 1.699997 ] [256, 144, 176]
post interpolate (256, 144, 176) (256, 144, 176)
rotate orient (256, 144, 176) (256, 144, 176)
move axis (176, 256, 144) (176, 256, 144)
file saved for data/KORA2460734.nii.gz
file saved for labels/KORA2460734.nii.gz
#original (

interpolate (245, 136, 163) (245, 136, 163)
[1.7013888 1.7013888 1.699997 ] [256, 144, 176]
post interpolate (256, 144, 176) (256, 144, 176)
rotate orient (256, 144, 176) (256, 144, 176)
move axis (176, 256, 144) (176, 256, 144)
file saved for data/KORA2460889.nii.gz
file saved for labels/KORA2460889.nii.gz
#original (288, 288, 160) (293, 427, 160)
reorient (288, 160, 288) (293, 160, 427)
interpolate (245, 136, 163) (249, 136, 242)
[1.7013888 1.7013888 1.699997 ] [256, 144, 176]
post interpolate (256, 144, 176) (260, 144, 189)
rotate orient (256, 144, 176) (260, 144, 189)
move axis (176, 256, 144) (189, 260, 144)
orig and seg shape does not match
rogue volume:  ['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2461184/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_13.nii.gz', '/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2461184/comp_mask.nii']
#original (288, 288, 160) (293, 428, 160)
reorient (288, 160, 288) (293, 160, 428)
interpolat

post interpolate (256, 144, 176) (260, 144, 189)
rotate orient (256, 144, 176) (260, 144, 189)
move axis (176, 256, 144) (189, 260, 144)
orig and seg shape does not match
rogue volume:  ['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2461349/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9.nii.gz', '/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2461349/comp_mask.nii']
#No such file or no access: '/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2462235/comp_mask.nii'
rogue volume:  ['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2462235/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_13.nii.gz', '/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2462235/comp_mask.nii']
#original (288, 288, 160) (293, 427, 161)
reorient (288, 160, 288) (293, 161, 427)
interpolate (245, 136, 163) (249, 137, 242)
[1.7013888 1.7013888 1.699997 ] [256, 144, 176]
post interpolate 

#original (288, 288, 160) (293, 426, 161)
reorient (288, 160, 288) (293, 161, 426)
interpolate (245, 136, 163) (249, 137, 241)
[1.7013888 1.7013888 1.7000046] [256, 144, 176]
post interpolate (256, 144, 176) (260, 145, 189)
rotate orient (256, 144, 176) (260, 145, 189)
move axis (176, 256, 144) (189, 260, 145)
orig and seg shape does not match
rogue volume:  ['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2461599/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_13.nii.gz', '/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2461599/comp_mask.nii']
#original (288, 288, 160) (293, 428, 160)
reorient (288, 160, 288) (293, 160, 428)
interpolate (245, 136, 163) (249, 136, 242)
[1.7013888 1.7013888 1.699997 ] [256, 144, 176]
post interpolate (256, 144, 176) (260, 144, 189)
rotate orient (256, 144, 176) (260, 144, 189)
move axis (176, 256, 144) (189, 260, 144)
orig and seg shape does not match
rogue volume:  ['/home/abhijit/nas_drive/Data_WholeBody/

#original (288, 288, 160) (293, 429, 161)
reorient (288, 160, 288) (293, 161, 429)
interpolate (245, 136, 163) (249, 137, 243)
[1.7013888 1.7013888 1.699997 ] [256, 144, 176]
post interpolate (256, 144, 176) (260, 145, 189)
rotate orient (256, 144, 176) (260, 145, 189)
move axis (176, 256, 144) (189, 260, 145)
orig and seg shape does not match
rogue volume:  ['/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/KORA2462054/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_13.nii.gz', '/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/KORA2462054/comp_mask.nii']
#original (288, 288, 160) (288, 288, 160)
reorient (288, 160, 288) (288, 160, 288)
interpolate (245, 136, 163) (245, 136, 163)
[1.7013888 1.7013888 1.699997 ] [256, 144, 176]
post interpolate (256, 144, 176) (256, 144, 176)
rotate orient (256, 144, 176) (256, 144, 176)
move axis (176, 256, 144) (176, 256, 144)
file saved for data/KORA2462374.nii.gz
file saved for labels/KORA2462374.nii.gz
#original 

In [ ]:
orientation = 'AXI'

# KORA
vol = 'KORA2460371'
paths_orig = '/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_all/KORA_Nifti/'+vol+'/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nii.gz'
paths_seg = '/home/abhijit/nas_drive/Data_WholeBody/KORA/KORA_segs/ROI_liver_spleen_Daniel/'+vol+'/comp_mask.nii'
# paths_orig = '/home/abhijit/Jyotirmay/thesis/punet/processedData/data/'+vol+'.nii.gz' 
# paths_seg = '/home/abhijit/Jyotirmay/thesis/punet/processedData/labels/'+vol+'.nii.gz' 

# paths_orig = '/home/abhijit/Jyotirmay/thesis/punet/processedData/data/100178_3D_GRE_TRA_opp.nii.gz' 
# paths_seg = '/home/abhijit/Jyotirmay/thesis/punet/processedData/labels/100178_3D_GRE_TRA_opp.nii.gz' 

file_paths = [paths_orig, paths_seg]
volume_k, labelmap_k, header_k = load_data(file_paths)
print('actual shape', volume_k.shape)
volume_k, labelmap_k = reorient(volume_k, labelmap_k, header_k)
print('after orient', volume_k.shape)
steps = header_k['pixdim'][1:4]
print(steps)
volume_k = do_interpolate(volume_k, steps)
labelmap_k = do_interpolate(labelmap_k, steps, True)
print('after interpolation', volume_k.shape)
target_dim = find_nearest(volume_k.shape)
volume_k, labelmap_k = post_interpolate(volume_k, labelmap_k, target_shape=target_dim)
print('after post interpolate', volume_k.shape)
volume_k, labelmap_k = rotate_orientation(volume_k, labelmap_k, orientation)
print('after rotation', volume_k.shape, orientation)
volume_k = normalise_data(volume_k)
print('after normalise data', volume_k.shape)
labelmap_k = np.moveaxis(labelmap_k, 2, 0)
volume_k = np.moveaxis(volume_k, 2, 0)
print('after move axis', volume_k.shape)
# volume_k, labelmap_k = remove_black(volume_k, labelmap_k)
# volume_k, labelmap_k = reduce_slices(volume_k, labelmap_k)


# NAKO
vol = '100178'
paths_orig = '/home/abhijit/nas_drive/Data_WholeBody/NAKO/NAKO_200/MRI/'+vol+'/'+vol+'_3D_GRE_TRA_opp/3D_GRE_TRA_opp_3D_GRE_TRA_2.nii.gz'
paths_seg = paths_orig
file_paths = [paths_orig, paths_seg]
volume_n, labelmap_n, header_n = load_data(file_paths)
volume_n, labelmap_n = reorient(volume_n, labelmap_n, header_n)
steps = header_n['pixdim'][1:4]
print(steps)
volume_n = do_interpolate(volume_n, steps)
# labelmap_n = do_interpolate(labelmap_n, steps, True)
target_dim = find_nearest(volume_n.shape)
volume_n, labelmap_n = post_interpolate(volume_n, labelmap_n, target_shape=target_dim)
volume_n, labelmap_n = rotate_orientation(volume_n, labelmap_n, orientation)
volume_n = normalise_data(volume_n)
labelmap_n = np.moveaxis(labelmap_n, 2, 0)
volume_n = np.moveaxis(volume_n, 2, 0)
# volume_n, labelmap_n = remove_black(volume_n, labelmap_n)
# volume_n, labelmap_n = reduce_slices(volume_n, labelmap_n)

# # UKB
# paths_orig = '/home/abhijit/Jyotirmay/ukbiobank/1000698_20201_2_0/Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz'
# paths_seg = paths_orig
# file_paths = [paths_orig, paths_seg]
# volume_u, labelmap_u, header_u = load_data(file_paths)
# volume_u, labelmap_u = reorient(volume_u, labelmap_u, header_u)
# steps = header_u['pixdim'][1:4]
# print(steps)
# volume_u = do_interpolate(volume_u, steps)
# # labelmap_u = do_interpolate(labelmap_u, steps, True)
# target_dim = find_nearest(volume_u.shape)
# volume_u, labelmap_u = post_interpolate(volume_u, labelmap_u, target_shape=target_dim)
# volume_u, labelmap_u = rotate_orientation(volume_u, labelmap_u, orientation)
# volume_u = normalise_data(volume_u)
# volume_u, labelmap_u = remove_black(volume_u, labelmap_u)
# # volume_u, labelmap_u = reduce_slices(volume_u, labelmap_u)


In [ ]:
idxkora = 60
idxnako = 60
idxukb = 35

print('kora', volume_k.shape, labelmap_k.shape)
plt.imshow(volume_k[idxkora,:, :], cmap='gray')
plt.show()

plt.imshow(labelmap_k[idxkora,:, :], cmap='gray')
plt.show()

print('nako', volume_n.shape)
plt.imshow(volume_n[idxnako, :,:], cmap='gray')
plt.show()

# print('UKB', volume_u.shape)
# plt.imshow(volume_u[idxukb,:,:], cmap='gray')
# plt.show()

In [6]:
import glob
path = "/home/abhijit/nas_drive/Data_WholeBody/UKBiobank/body/body_nifti/*"
x = []
for p in glob.glob(path):
    x.append(p.split('/')[-1])

print(x)

['1000410_20201_2_0', '1000698_20201_2_0', '1000857_20201_2_0', '1000870_20201_2_0', '1002272_20201_2_0', '1002359_20201_2_0', '1002876_20201_2_0', '1003082_20201_2_0', '1004466_20201_2_0', '1004543_20201_2_0', '1004760_20201_2_0', '1004985_20201_2_0', '1005188_20201_2_0', '1005379_20201_2_0', '1005645_20201_2_0', '1005691_20201_2_0', '1006276_20201_2_0', '1006486_20201_2_0', '1006931_20201_2_0', '1007005_20201_2_0', '1007156_20201_2_0', '1007400_20201_2_0', '1007576_20201_2_0', '1007659_20201_2_0', '1007864_20201_2_0', '1008067_20201_2_0', '1008133_20201_2_0', '1008283_20201_2_0', '1008312_20201_2_0', '1008800_20201_2_0', '1009220_20201_2_0', '1009657_20201_2_0', '1010111_20201_2_0', '1010355_20201_2_0', '1010361_20201_2_0', '1010424_20201_2_0', '1010532_20201_2_0', '1010630_20201_2_0', '1011155_20201_2_0', '1011301_20201_2_0', '1011409_20201_2_0', '1012512_20201_2_0', '1012789_20201_2_0', '1013250_20201_2_0', '1013391_20201_2_0', '1014447_20201_2_0', '1014488_20201_2_0', '1014541_202